# Spreadsheets on GoogleDocs

- [HFRadar](https://docs.google.com/spreadsheets/d/11hWfIr4lrKP-RviEwSio6dZgnm0oUFUII9WChPDzeAw/edit#gid=62536835)
- [Stations](https://docs.google.com/spreadsheets/d/1ECXoa43uq9Gr8REZF-7EGlxNa1mkP7-4YturX0nFK7s/edit#gid=0)
- [Gliders](https://docs.google.com/spreadsheets/d/13e906UHl6ibF2lx1F_mu_9vBVptVkESM7alw5-FRJdI/edit#gid=65941256)

# Instantiate the map centered on SECOORA

In [ ]:
import folium

lon, lat = -86.276, 30.935 

tiles = ('http://services.arcgisonline.com/arcgis/rest/'
         'services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}')

mapa = folium.Map(location=[lat, lon], tiles=tiles,
                  attr='ESRI', zoom_start=5)

# Create the GIS database from the spreadsheets

In [ ]:
import os

fname = 'hfradar/hfradar.geojson'
if os.path.isfile(fname):
    os.unlink(fname)

%pushd hfradar/

%run convert.py
%popd

In [ ]:
fname = 'stations/'
if os.path.isfile(fname):
    os.unlink(fname)

%pushd stations/
%run convert.py
%popd

In [ ]:
fname = 'gliders//'
if os.path.isfile(fname):
    os.unlink(fname)

%pushd stations/
%run convert.py
%popd

# Stations

In [ ]:
import json

with open('stations/stations.geojson') as f:
    stations = json.load(f)


icon_size = (14, 14)
layer_stations = folium.FeatureGroup()
for feature in stations['features']:
    lon, lat = feature['geometry']['coordinates']
    icon_url = feature['properties']['icon']
    popup = feature['properties']['popupcontent']
    
    icon = folium.features.CustomIcon(icon_url,
                                      icon_size=(14, 14))
    
    marker = folium.map.Marker([lat, lon], icon=icon,
                               popup=folium.map.Popup(popup))
    layer_stations.add_children(marker)

layer_stations.layer_name = 'Stations'
mapa.add_children(layer_stations)

# HF-Radar

In [ ]:
with open('hfradar/hfradar.geojson') as f:
    hfradar = json.load(f)

    
style_function = lambda feature: dict(fillColor='#DEFFDE',
                                      color='#AECCAE',
                                      weight=1,
                                      opacity=0.6,
                                      dashArray='0')

layer_hfradar = folium.FeatureGroup()
for feature in hfradar['features']:
    if feature['geometry']['type'] == 'Point':
        lon, lat = feature['geometry']['coordinates']
        icon_url = feature['properties']['icon']
        popup = feature['properties']['popupcontent']
        
        icon = folium.features.CustomIcon(icon_url,
                                          icon_size=(14, 14))
        marker = folium.map.Marker([lat, lon], icon=icon,
                                   popup=folium.map.Popup(popup))
        layer_hfradar.add_children(marker)
    elif feature['geometry']['type'] == 'Polygon':
        gjson = folium.features.GeoJson(feature,
                                        style_function=style_function)
        layer_hfradar.add_children(gjson)

layer_hfradar.layer_name = 'HFRadar'
mapa.add_children(layer_hfradar)

# AUVs

In [ ]:
import json

with open('gliders/gliders.geojson') as f:
    gliders = json.load(f)


icon_size = (14, 14)
layer_gliders = folium.FeatureGroup()
for feature in gliders['features']:
    lon, lat = feature['geometry']['coordinates']
    icon_url = feature['properties']['icon']
    popup = feature['properties']['popupcontent']
    
    icon = folium.features.CustomIcon(icon_url,
                                      icon_size=(14, 14))
    
    marker = folium.map.Marker([lat, lon], icon=icon,
                               popup=folium.map.Popup(popup))
    layer_gliders.add_children(marker)

layer_gliders.layer_name = 'AUVs'
mapa.add_children(layer_gliders)

# Numerical Models

In [ ]:
import iris
import warnings
from gridgeo import GridGeo
from netCDF4 import Dataset


models = ['http://omgsrv1.meas.ncsu.edu:8080/thredds/dodsC/fmrc/sabgom/SABGOM_Forecast_Model_Run_Collection_best.ncd',
          'http://crow.marine.usf.edu:8080/thredds/dodsC/FVCOM-Nowcast-Agg.nc']

layer_models = folium.FeatureGroup()
for model in models:
    with Dataset(model) as nc:
        try:
            grid = GridGeo(nc)
        except ValueError:
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                cube = iris.load_cube(model, 'sea_water_potential_temperature')
                grid = GridGeo(cube)
        gj = folium.GeoJson(grid.outline.__geo_interface__)
        gj.add_children(folium.Popup(getattr(nc, 'summary', url)))
        layer_models.add_children(gj)

layer_models.layer_name = 'Models'
mapa.add_children(layer_models)

In [ ]:
mapa.add_children(folium.LayerControl())

mapa